In [13]:
!pip install -U langchain_experimental openai --use-deprecated=legacy-resolver


Requirement already up-to-date: langchain_experimental in /usr/local/lib/python3.10/dist-packages (0.0.52)
Requirement already up-to-date: openai in /usr/local/lib/python3.10/dist-packages (1.12.0)


In [14]:
import os
os.environ['OPENAI_API_KEY']='--------------------------------'

In [16]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_PREFIX, SYNTHETIC_FEW_SHOT_SUFFIX

In [17]:
class database_llm(BaseModel):
  App : str
  Translated_Review : str
  Sentiment : str

In [18]:
examples=[
    {'example':"App: 10 Best Foods for You, Translated_Review: Thank you! Great app!! Add arthritis, eyes, immunity, kidney/liver detox foods please, Sentiment: Positive"},
    {'example':"App: 11800 Contacts - Lens Store, Translated_Review: So easy app! Made ordering contacts breeze! Great prices too!, Sentiment: Positive"},
    {'example':"App: 1LINE – One Line with One Touch, Translated_Review: Terrible amount ads covering content making unpleasant play. Avoid, Sentiment: Negative"},
    {'example':"App: Apartments.com Rental Search, Translated_Review: Crashes constantly, Sentiment: Nuetral"},

]

In [19]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=['example'], template='{example}')
prompt_template= FewShotPromptTemplate(
    examples=examples,
    prefix= SYNTHETIC_FEW_SHOT_PREFIX,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=['subject','extra'],
    example_prompt=OPENAI_TEMPLATE,
)

In [20]:
synthetic_data= create_openai_data_generator(
    output_schema= database_llm,
    llm= ChatOpenAI(temperature=1),
    prompt= prompt_template,
)

In [21]:
result=synthetic_data.generate(
    subject='app_review',
    extra='the app name should be choosen from any playstore app and make reviews with sentiments like Positive, Negative and Nuetral',
    runs = 50,
)

In [22]:
result

[database_llm(App='Newest Fashion Trends', Translated_Review='Love this app! So many trendy outfit ideas to choose from. Highly recommend!', Sentiment='Positive'),
 database_llm(App='Google Play Store', Translated_Review='Best app for downloading and updating Android apps. Simple and user-friendly interface.', Sentiment='Positive'),
 database_llm(App='Instagram', Translated_Review='Great app for sharing photos and videos with friends. Easy to use and navigate.', Sentiment='Positive'),
 database_llm(App='Spotify Music', Translated_Review='Love the variety of music available on this app. Super easy to create playlists and discover new artists.', Sentiment='Positive'),
 database_llm(App='WhatsApp Messenger', Translated_Review="I find this app very convenient for messaging my friends and family. It's reliable and easy to use.", Sentiment='Positive'),
 database_llm(App='Facebook', Translated_Review="I'm not happy with the recent updates on this app. It's become very slow and buggy.", Sentim

In [23]:
import pandas as pd
synthetic_data = []
for item in result:
    synthetic_data.append({
        'App': item.App,
        'Translated_Review': item.Translated_Review,
        'Sentiment': item.Sentiment,
        })
synthetic_df = pd.DataFrame(synthetic_data)
print(type(synthetic_df))
synthetic_df

<class 'pandas.core.frame.DataFrame'>


,App,Translated_Review,Sentiment
0,Newest Fashion Trends,Love this app! So many trendy outfit ideas to ...,Positive
1,Google Play Store,Best app for downloading and updating Android ...,Positive
2,Instagram,Great app for sharing photos and videos with f...,Positive
3,Spotify Music,Love the variety of music available on this ap...,Positive
4,WhatsApp Messenger,I find this app very convenient for messaging ...,Positive
5,Facebook,I'm not happy with the recent updates on this ...,Negative
6,Twitter,Love how easy it is to follow conversations an...,Positive
7,Instagram,I enjoy using Instagram to share photos and co...,Positive
8,YouTube,I spend hours watching videos on YouTube. It's...,Positive
9,WhatsApp,I love using WhatsApp to stay connected with m...,Positive


In [25]:
synthetic_df.to_csv('synthetic_data_LLM1.csv', index=False)